In [ ]:
! pip install tabulate

In [36]:
import pymysql
import math

connection = pymysql.connect(
    host = "localhost",
    user = "root",
    password = "",
    database="tennis_db"

)

mycursor = connection.cursor()


In [3]:
from tabulate import tabulate

In [ ]:
#List all competitions along with their category name

mycursor.execute("select competitions.competition_name,categories.category_name from competitions INNER JOIN categories ON competitions.category_id = categories.category_id")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Count the number of competitions in each category

mycursor.execute("select categories.category_name, count(competitions.competition_name) AS CompetitionsCount from competitions LEFT JOIN categories ON competitions.category_id = categories.category_id GROUP BY categories.category_name")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Find all competitions of type 'doubles'

mycursor.execute("select competition_name,type from competitions where type = 'doubles'")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Get competitions that belong to a specific category (e.g., ITF Men)

mycursor.execute("select categories.category_name, competitions.competition_name from competitions LEFT JOIN categories ON competitions.category_id = categories.category_id where categories.category_name='ITF Men'")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Identify parent competitions and their sub-competitions

mycursor.execute("select p.parent_id, GROUP_CONCAT(p.competition_id SEPARATOR ', ') AS child_competition_id from competitions p where p.parent_id != '' group by p.parent_id;")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Analyze the distribution of competition types by category - Need to be clarified

mycursor.execute("select categories.category_name, competitions.type,competitions.competition_name from competitions LEFT JOIN categories ON competitions.category_id = categories.category_id ORDER BY categories.category_name")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#List all competitions with no parent (top-level competitions)

mycursor.execute("SELECT * FROM competitions WHERE parent_id = ''")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#List all venues along with their associated complex name

mycursor.execute("select venues.venue_name, complexes.complex_name from venues LEFT JOIN complexes ON venues.complex_id = complexes.complex_id ORDER BY venues.venue_name")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Count the number of venues in each complex

mycursor.execute("select complexes.complex_name, count(venues.venue_name) AS VenueCount from complexes LEFT JOIN venues ON venues.complex_id = complexes.complex_id GROUP BY complexes.complex_name")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Get details of venues in a specific country (e.g., Chile)

mycursor.execute("select complexes.complex_name,venues.venue_name,venues.city_name,venues.country_name from venues LEFT JOIN complexes ON venues.complex_id = complexes.complex_id where venues.country_name='Chile'")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Identify all venues and their timezones

mycursor.execute("select venue_name, city_name, country_name,timezone from venues ORDER BY country_name ASC")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Find complexes that have more than one venue

# mycursor.execute("select complexes.complex_name,venues.venue_name from venues LEFT JOIN complexes ON venues.complex_id = complexes.complex_id ORDER BY complexes.complex_name DESC")
mycursor.execute("select cmp.complex_name,count(venues.venue_name) as venues_count from complexes cmp LEFT JOIN venues ON venues.complex_id = cmp.complex_id where (select count(venues.venue_name) as venues_count from complexes LEFT JOIN venues ON venues.complex_id = complexes.complex_id where complexes.complex_id = cmp.complex_id group by complexes.complex_name) > 1 group by cmp.complex_name order by venues_count desc")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#List venues grouped by country 

# mycursor.execute("select venue_name,country_name from venues group by country_name")
mycursor.execute("select country_name, GROUP_CONCAT(venue_name SEPARATOR ',') as venues from venues group by country_name order by country_name")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Find all venues for a specific complex (e.g., Nacional)

mycursor.execute("SELECT complexes.complex_name,venues.venue_name from complexes LEFT JOIN venues ON venues.complex_id = complexes.complex_id WHERE complexes.complex_name='Nacional'")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Get all competitors with their rank and points.

mycursor.execute("SELECT competitors.name,competitor_rankings.rank,competitor_rankings.points from competitor_rankings LEFT JOIN competitors ON competitors.competitor_id = competitor_rankings.competitor_id")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Find competitors ranked in the top 5 -- need to clarified - query is based on rank only or points also to be considered.

mycursor.execute("SELECT competitors.name,competitor_rankings.rank,competitor_rankings.points from competitors JOIN competitor_rankings ON competitors.competitor_id = competitor_rankings.competitor_id where competitor_rankings.rank < 6 order by competitor_rankings.rank ASC")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#List competitors with no rank movement (stable rank)

mycursor.execute("SELECT c.name, cr.movement FROM competitor_rankings cr join competitors c on c.competitor_id = cr.competitor_id WHERE movement = 0")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Get the total points of competitors from a specific country (e.g., Croatia)

mycursor.execute("SELECT competitors.country,count(competitors.name) AS Headcount, sum(competitor_rankings.points) AS TotalPoint from competitors JOIN competitor_rankings ON competitors.competitor_id = competitor_rankings.competitor_id where competitors.country ='Argentina'")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Count the number of competitors per country

mycursor.execute("SELECT country,count(*) AS Number from competitors group by country order by country ASC")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#Find competitors with the highest points in the current week -- need to check

mycursor.execute("SELECT competitors.name,competitor_rankings.week, competitor_rankings.points from competitors INNER JOIN competitor_rankings ON competitors.competitor_id = competitor_rankings.competitor_id where points = (select MAX(competitor_rankings.points) from competitor_rankings)")
connection.commit()
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
mycursor.close()
connection.close()